# 增强--对每类物品单独设置目标数量

In [3]:
from pyecharts.globals import CurrentConfig, NotebookType, OnlineHostType
CurrentConfig.ONLINE_HOST = "https://assets.pyecharts.org/assets/"
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
from IPython.display import display, clear_output
import os
import datetime
import time
import csv
import sympy
import pandas as pd
from pyecharts.charts import Bar,Grid,Pie
from pyecharts import options as opts
from class_aug import directory_check,get_files,class_num,log_file,if_balanced,class_sort,image_sort,get_aug_list,\
                       augmentation,draw,draw_show,goal_class,find_class,if_satisfy,log_file,round_time,class_percent,\
                       pyecharts_line_bar,pyecharts_pie,pyecharts_bar_stack

In [4]:
# the path of source annotations and images

xml_source_path = 'D:/lab/boruishi/Annotations/'
image_source_path = 'D:/lab/boruishi/JPEGImages/'

# the path to save new annotations and images

xml_path = 'D:/lab/boruishi/augmentation_5/Annotations/'
image_path = 'D:/lab/boruishi/augmentation_5/JPEGImages/'

# the log filepath

log_path = os.path.join(xml_path, os.path.pardir+"/"+"log.csv")

  ## 对原数据集里的数据进行统计、展示

In [5]:
filenames = get_files(xml_source_path)
source_class_names, source_num_objs, source_source_list = class_num(filenames,xml_source_path)
source_file_name = list(source_source_list.keys())
#-----------------------画图----------------------------
x_data = list(source_num_objs.keys())
v1 = list(source_num_objs.values())
average,average_list = class_percent(source_num_objs)
v2 = [i*100 for i in average_list]
grid = pyecharts_line_bar(x_data, v1, v2, source_num_objs)
pie = pyecharts_pie(x_data, v1)

grid.render_notebook()
pie.render_notebook()
#-----------------------画图----------------------------
df = pd.DataFrame([list(source_num_objs.keys()),list(source_num_objs.values())])
pd.set_option('display.max_columns',200)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,phone,umbrella,battery,mirror,plastic,laggage,bottle,knife,cookknife,foldknife,glass,fixture,gun,handcuffs,oil,baggedliqiud,cans,firecracker,slandy,nunchucks,spray,grenade,handguard,lunchbox,firework,electric,flour,hammer,wrench,scissors,expdevice,catapult
1,986,589,677,412,1565,238,1564,888,119,536,558,369,369,96,134,147,229,109,98,94,184,244,241,58,257,90,64,132,106,120,86,253


## 输入每个类别的目标数量

In [6]:
"""
输入完一个类别的目标数量后回车跳到下一个类别，如果该类别不进行增强，则直接不输入数字回车跳过
"""
goal_num_obj = {}
for obj in source_class_names:
    a = input('请输入'+str(obj)+'类别的目标数量：')
    goal_num_obj[obj] = a
print("目标数量：", goal_num_obj)

'\n输入完一个类别的目标数量后回车跳到下一个类别，如果该类别不进行增强，则直接不输入数字回车跳过\n'

请输入phone类别的目标数量：1000
请输入umbrella类别的目标数量：700
请输入battery类别的目标数量：900
请输入mirror类别的目标数量：
请输入plastic类别的目标数量：
请输入laggage类别的目标数量：900
请输入bottle类别的目标数量：
请输入knife类别的目标数量：
请输入cookknife类别的目标数量：
请输入foldknife类别的目标数量：
请输入glass类别的目标数量：900
请输入fixture类别的目标数量：
请输入gun类别的目标数量：
请输入handcuffs类别的目标数量：
请输入oil类别的目标数量：
请输入baggedliqiud类别的目标数量：
请输入cans类别的目标数量：
请输入firecracker类别的目标数量：
请输入slandy类别的目标数量：
请输入nunchucks类别的目标数量：
请输入spray类别的目标数量：
请输入grenade类别的目标数量：
请输入handguard类别的目标数量：
请输入lunchbox类别的目标数量：
请输入firework类别的目标数量：
请输入electric类别的目标数量：
请输入flour类别的目标数量：
请输入hammer类别的目标数量：
请输入wrench类别的目标数量：
请输入scissors类别的目标数量：
请输入expdevice类别的目标数量：
请输入catapult类别的目标数量：
目标数量： {'phone': '1000', 'umbrella': '700', 'battery': '900', 'mirror': '', 'plastic': '', 'laggage': '900', 'bottle': '', 'knife': '', 'cookknife': '', 'foldknife': '', 'glass': '900', 'fixture': '', 'gun': '', 'handcuffs': '', 'oil': '', 'baggedliqiud': '', 'cans': '', 'firecracker': '', 'slandy': '', 'nunchucks': '', 'spray': '', 'grenade': '', 'handguard': '', 'lunchbox

## 对输入的目标数据进行处理

In [7]:
directory_check(xml_path, image_path, xml_source_path, image_source_path)

# get and save the source data of the dataset
filenames = get_files(xml_path)
source_class_names, source_num_objs, source_source_list = class_num(filenames,xml_path)
source_file_name = list(source_source_list.keys())
class_list = class_sort(source_num_objs)
image_list = image_sort(class_list, source_source_list)
# from input get the final goal number of every class
final_goal_obj_num, no_goal_class = goal_class(goal_num_obj, class_list)
print(final_goal_obj_num)
print(no_goal_class)

clear the exist xml_path !
clear the exist image_path !
{16: ['laggage', '900'], 7: ['glass', '900'], 6: ['umbrella', '700'], 5: ['battery', '900'], 3: ['phone', '1000']}
['lunchbox', 'flour', 'expdevice', 'electric', 'nunchucks', 'handcuffs', 'slandy', 'wrench', 'firecracker', 'cookknife', 'scissors', 'hammer', 'oil', 'baggedliqiud', 'spray', 'cans', 'handguard', 'grenade', 'catapult', 'firework', 'fixture', 'gun', 'mirror', 'foldknife', 'knife', 'bottle', 'plastic']


## 进行增强

In [8]:
turns = 0
start_time = datetime.datetime.now()
# augment classes in turns
for key, value in final_goal_obj_num.items():
    print("开始增强类别：", value[0])
    # get the image list of one class
    list1 = find_class(key, image_list, no_goal_class)
    print(list1)
    # some parameters
    times = 0
    a = 0
    
    """
    -------------------------------------------------------------------------------------------------------------------
    如果选择只增强含有单类别的图片，则选择下列两行代码，如果某类别不存在只含有该类别的图片，则转为增强含有多类别的图片
    round_one_obj = float('inf')
    round_two_obj = float('inf')
    """
    
    """
    --------------------------------------------------------------------------------
    如果选择按比例增强含有单类别，两个类别以及多类别的图片，则选择以下三行代码
    list_round = round_time(list1, value)
    round_one_obj = list_round[0]
    round_two_obj = list_round[1]
    """
    round_one_obj = float('inf')
    round_two_obj = float('inf')
    
    # if the class number doesn't meet the requirement, continue the cycle
    while a == 0:
        clear_output(wait=True)       
        filenames = get_files(xml_path)
        # get quantity of every class
        class_names, num_objs, source_list = class_num(filenames,xml_path)
        # write file log
        log_file(turns, source_class_names, num_objs, log_path)
        a = if_satisfy(value, num_objs)
        if a == 0:
            clear_output()
            if list1[0] and times <= round_one_obj:
                augmentation(list1[0], times, source_file_name, xml_path, image_path)
#                 print('aaaaaaaaaaaaa')
            elif list1[1] and times <= round_one_obj + round_two_obj:
                augmentation(list1[1], times, source_file_name, xml_path, image_path)
#                 print("bbbbbbbbbbbbbbbbbbbbb")
            elif list1[2]:
                augmentation(list1[2], times, source_file_name, xml_path, image_path)
#                 print('ccccccccccccccccc')
            else:
                list2 = [a for a in list1 if a ]
                for obj in list2:
                    augmentation(obj, times, source_file_name, xml_path, image_path)
#                     print("eeeeeeeeeeeeee")
            turns = turns + 1
            times = times + 1
            print("times:", times)
 #-----------------------------------------画图----------------------------------------------------------------------             
        x_data = list(num_objs.keys())
        bar_2 = pyecharts_bar_stack(x_data,source_num_objs, num_objs )
        bar_2.render_notebook()
 #--------------------------------------画图--------------------------------------------------------------------------
        time.sleep(2)
print('finish augmentation')
# draw(source_num_objs, num_objs,xml_path)
end_time = datetime.datetime.now()
spend_time = (end_time - start_time).seconds
print("time spend: ", spend_time / 60, "min")
print("设定的目标类别数量：",final_goal_obj_num)
with open(log_path, 'a+', newline='') as f:
    csv_write = csv.writer(f)
    data_row = ['time_spend', spend_time / 60, 'min']
    csv_write.writerow(data_row)
x_data = list(num_objs.keys())
v1 = list(num_objs.values())
pie = pyecharts_pie(x_data, v1)
pie.render_notebook()

20170605_3941 out of picture
sf20190401_1330 out of picture
times: 3


KeyboardInterrupt: 